In [1]:
# **Import essential libraries**
import os  # For handling file paths
import numpy as np  # For numerical computations
import matplotlib.pyplot as plt  # For data visualization
import rasterio  # For working with raster data
from rasterio.windows import Window  # Optional: Reading specific raster windows


In [ ]:
# Set the path to the data directory
data_dir = r"C:\Users\ASUA\OneDrive - Institut Teknologi Bandung\Geo-X\Rock Lens\python\Dataset Final"

# Set the path to the point data
point_fn = os.path.join(data_dir, 'AOI_fix2.gpkg')
# Recursively collect all TIFF files in subdirectories
raster_fns = []
for root, _, files in os.walk(data_dir):
    for file in files:
        if file.endswith('.tif') and 'xml' not in file:
            raster_fns.append(os.path.join(root, file))

# Print the results to verify
print(f"Point File: {point_fn}")
print("Raster Files:")
for fn in raster_fns:
    print(fn)


In [ ]:
import os
import rasterio
from rasterio.enums import Resampling

def resize_tiff_files(raster_fns, new_width, new_height, output_dir):
    """
    Resize multiple TIFF files, handling both single-band and multi-band TIFFs.
    """
    os.makedirs(output_dir, exist_ok=True)

    for input_tiff in raster_fns:
        try:
            with rasterio.open(input_tiff) as src:
                # Check the number of bands in the TIFF
                band_count = src.count
                print(f"Processing {input_tiff}: {band_count} band(s)")

                # Read all bands (if multi-band)
                data = src.read(
                    out_shape=(band_count, new_height, new_width),
                    resampling=Resampling.nearest  # Nearest neighbor to maintain values
                )

                # Update the metadata profile
                profile = src.profile
                profile.update({
                    'width': new_width,
                    'height': new_height,
                    'transform': src.transform * src.transform.scale(
                        src.width / new_width,
                        src.height / new_height
                    ),
                    'count': band_count  # Update the number of bands
                })

                # Define output path
                output_tiff = os.path.join(output_dir, os.path.basename(input_tiff))

                # Save the resized TIFF with all bands
                with rasterio.open(output_tiff, 'w', **profile) as dst:
                    dst.write(data)

                print(f"Resized {input_tiff} -> {output_tiff}")

        except Exception as e:
            print(f"Failed to process {input_tiff}: {e}")

# Define input and output directories, and new dimensions
output_dir = r'C:\Users\ASUA\OneDrive - Institut Teknologi Bandung\Geo-X\Rock Lens\python\Dataset Final\resize_tiffs'
new_width = 694
new_height = 1453

# Call the resize function
resize_tiff_files(raster_fns, new_width, new_height, output_dir)



In [ ]:
import os
import rasterio
from rasterio.transform import from_bounds
import matplotlib.pyplot as plt

def adjust_tiff_coordinates(input_tiff, output_tiff, lat_min, lat_max, lon_min, lon_max):
    """
    Fungsi untuk mengubah koordinat dan CRS dari TIFF file menjadi EPSG:4326 (latitude/longitude).
    
    Parameters:
    - input_tiff: Path ke TIFF file input.
    - output_tiff: Path ke TIFF file output.
    - lat_min, lat_max: Batas latitude (min dan max).
    - lon_min, lon_max: Batas longitude (min dan max).
    """
    with rasterio.open(input_tiff) as src:
        data = src.read()  # Baca semua band
        profile = src.profile  # Salin metadata asli

        # Buat transform baru berdasarkan latitude dan longitude
        transform = from_bounds(lon_min, lat_min, lon_max, lat_max, src.width, src.height)

        # Perbarui metadata dengan transformasi baru dan CRS WGS84
        profile.update({
            'transform': transform,
            'crs': 'EPSG:4326'  # WGS84 (latitude/longitude)
        })

        # Simpan TIFF baru dengan transformasi yang telah disesuaikan
        with rasterio.open(output_tiff, 'w', **profile) as dst:
            dst.write(data)

    print(f"TIFF file '{os.path.basename(input_tiff)}' telah disesuaikan dan disimpan ke: {output_tiff}")

def process_all_tiffs(raster_fns, output_dir, lat_min, lat_max, lon_min, lon_max):
    """
    Fungsi untuk mengubah koordinat semua TIFF file dalam daftar raster_fns.
    
    Parameters:
    - raster_fns: Daftar path TIFF files yang akan diproses.
    - output_dir: Direktori untuk menyimpan TIFF files yang sudah disesuaikan.
    - lat_min, lat_max, lon_min, lon_max: Batas latitude dan longitude untuk semua file.
    """
    os.makedirs(output_dir, exist_ok=True)  # Buat direktori jika belum ada

    for tiff_path in raster_fns:
        output_tiff = os.path.join(output_dir, os.path.basename(tiff_path))
        adjust_tiff_coordinates(tiff_path, output_tiff, lat_min, lat_max, lon_min, lon_max)

    print("Semua TIFF files telah selesai diproses.")

# Contoh penggunaan
tiff_dir = r"C:\Users\ASUA\OneDrive - Institut Teknologi Bandung\Geo-X\Rock Lens\python\Dataset Final\resized_tiffs"
output_dir = r"C:\Users\ASUA\OneDrive - Institut Teknologi Bandung\Geo-X\Rock Lens\python\Dataset Final\tiffs_Final"

# Kumpulkan semua TIFF files di direktori
raster_fns = [os.path.join(tiff_dir, f) for f in os.listdir(tiff_dir) if f.endswith('.tif')]

# Batas latitude dan longitude
lat_min = 23.5  # 23° 30’ 00’’N -> 23.5°
lat_max = 25.25  # 25° 15’ 00’’N -> 25.25°
lon_min = 73.75  # 73° 45’ 00’’E -> 73.75°
lon_max = 74.5  # 74° 30’ 00’’E -> 74.5°

# Proses semua TIFF files
process_all_tiffs(raster_fns, output_dir, lat_min, lat_max, lon_min, lon_max)


In [ ]:
import geopandas as gpd
import os

# Set the path to the main data directory
data_dir = r"C:\Users\ASUA\OneDrive - Institut Teknologi Bandung\Geo-X\Rock Lens\python\Dataset Final"

# Define paths to the GPKG files
point_fn = os.path.join(data_dir, 'AOI_fix2.gpkg')  # Main GPKG with mineral data
none_points_gpkg_path = os.path.join(data_dir, 'None_points.gpkg')  # None points GPKG

# Load the mineral occurrence points from the main GPKG file
df_points = gpd.read_file(point_fn)

# Filter rows where the 'Tipe' column contains 'Porfiri' (case-insensitive)
Porphyry_points = df_points[df_points['Tipe'].str.contains('Porfiri', case=False, na=False)]

# Print the filtered DataFrame for Porphyry points
print("Porphyry Points:")
print(Porphyry_points.head())

# Load None points from the 'None_points.gpkg' file
none_points = gpd.read_file(none_points_gpkg_path)

# Print the None points DataFrame
print("\nNone Points:")
print(none_points.head())


In [ ]:
import os
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import geopandas as gpd

# Set the directory containing the TIFF files
tiff_porphyry = r"C:\Users\ASUA\OneDrive - Institut Teknologi Bandung\Geo-X\Rock Lens\python\Dataset Final\Porphyry_Final"

# Collect all TIFF file paths from the directory
raster_fns = [os.path.join(tiff_porphyry, f) for f in os.listdir(tiff_porphyry) if f.endswith('.tif')]

# Ensure Porphyry points are loaded correctly (from the previous cell)
try:
    Porphyry_points  # Check if the Porphyry points dataframe is available
except NameError:
    print("The 'Porphyry_points' variable is not defined. Make sure to run the cell with the GPKG data first.")
    Porphyry_points = None

# Set up the figure for plotting (4x4 grid)
fig, axes = plt.subplots(4, 4, figsize=(12, 18), constrained_layout=True)

# Loop through each TIFF file and plot it
for i, (tiff_path, ax) in enumerate(zip(raster_fns, axes.flatten())):
    try:
        # Open the TIFF file
        with rasterio.open(tiff_path) as src:
            band_count = src.count  # Check the number of bands

            # Handle different band counts
            if band_count == 1:
                # Single-band (grayscale)
                data = src.read(1)
                ax.imshow(data, extent=[src.bounds.left, src.bounds.right, 
                                        src.bounds.bottom, src.bounds.top], 
                          cmap='gray', 
                          vmin=np.nanpercentile(data, 5), 
                          vmax=np.nanpercentile(data, 95))
            
            elif band_count == 3:
                # Three-band (RGB)
                r = src.read(1)
                g = src.read(2)
                b = src.read(3)

                # Normalize the RGB data to [0, 1]
                rgb = np.dstack((r / r.max(), g / g.max(), b / b.max()))
                ax.imshow(rgb, extent=[src.bounds.left, src.bounds.right, 
                                       src.bounds.bottom, src.bounds.top])

            elif band_count == 4:
                # Four-band (RGBA)
                r = src.read(1)
                g = src.read(2)
                b = src.read(3)
                a = src.read(4)  # Alpha channel

                # Normalize RGBA data to [0, 1]
                rgba = np.dstack((r / r.max(), g / g.max(), b / b.max(), a / a.max()))
                ax.imshow(rgba, extent=[src.bounds.left, src.bounds.right, 
                                        src.bounds.bottom, src.bounds.top])

            # Plot Porphyry points on top if available
            if Porphyry_points is not None:
                Porphyry_points.plot(ax=ax, markersize=50, facecolor='blue', 
                                     edgecolor='red', marker='*', linewidth=1)

            # Set the title with the TIFF file name
            ax.set_title(os.path.basename(tiff_path), fontsize=8)

    except Exception as e:
        print(f"Failed to process {tiff_path}: {e}")
        ax.axis('off')  # Turn off the axis if the plot fails

# Hide any remaining empty axes
for ax in axes.flatten()[len(raster_fns):]:
    ax.axis('off')

# Display the plot
plt.show()
